In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix, csr_matrix
from implicit.als import AlternatingLeastSquares

# 데이터: 장바구니 담기 기준
data = pd.DataFrame({
    'user_id': [0, 1, 1, 2, 3, 3],
    'item_id': [101, 101, 102, 103, 102, 104],
    'cart_count': [1, 2, 1, 1, 3, 1]
})

# 인코딩
user_enc = LabelEncoder()
item_enc = LabelEncoder()
data['user_idx'] = user_enc.fit_transform(data['user_id'])
data['item_idx'] = item_enc.fit_transform(data['item_id'])

# 희소행렬 (user x item)
matrix = coo_matrix(
    (data['cart_count'], (data['user_idx'], data['item_idx']))
)

# ALS 학습
model = AlternatingLeastSquares(factors=10, iterations=15)
model.fit(matrix.tocsr())

# 추천 대상 유저
user_item_matrix = matrix.tocsr()
user_idx = 0
user_vector = csr_matrix(user_item_matrix[user_idx])

# 추천 수행
recommended = model.recommend(userid=user_idx, user_items=user_vector, N=3)
print("추천 결과:", recommended)

# 결과 출력
item_indices, scores = recommended
for item_idx, score in zip(item_indices, scores):
    print(f"item_idx: {item_idx}, 점수: {score:.4f}")

print(f'\n{user_idx}번 유저의 추천 상품')
original_item_ids = item_enc.inverse_transform(item_indices)
for item_id, score in zip(original_item_ids, scores):
    print(f"item_id: {item_id}, 점수: {score:.4f}")


C:\Users\EZENIC-213T\anaconda3\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

추천 결과: (array([1, 2, 3]), array([ 0.00430075,  0.00127119, -0.00413087], dtype=float32))
item_idx: 1, 점수: 0.0043
item_idx: 2, 점수: 0.0013
item_idx: 3, 점수: -0.0041

0번 유저의 추천 상품
item_id: 102, 점수: 0.0043
item_id: 103, 점수: 0.0013
item_id: 104, 점수: -0.0041
